In [18]:
import os
import cv2
import numpy as np
from  tqdm import tqdm
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

REBUILD_DATA = False

if torch.cuda.is_available():
    device = torch.device("cuda:0")
    print("runing on the GPU")
else:
    device = torch.device("cpu")
    print("runing on the cpu")

class DogVSCats():
    IMG_SIZE = 50
    CATS = "D:\KMUTT\Year 4th\ENE490 DL\Coding\PetImages\Cat"
    DOGS = "D:\KMUTT\Year 4th\ENE490 DL\Coding\PetImages\Dog"
    LABELS = {CATS: 0, DOGS: 1}

    training_data = []
    catcount = 0
    dogcount = 0

    def make_training_data(self):
        for label in self.LABELS:
            print(label)
            for f in tqdm(os.listdir(label)):
                try:
                    path = os.path.join(label, f)
                    img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
                    img = cv2.resize(img, (self.IMG_SIZE, self.IMG_SIZE))
                    # self.training_data.append([np.array(img), np.eye(2)[self.LABELS[label]]])
                    # self.training_data = np.array(self.training_data, dtype=object)
                    self.training_data.append([np.array(img), np.eye(2)[self.LABELS[label]]])

                    if label == self.CATS:
                        self.catcount += 1
                    elif label == self.DOGS:
                        self.dogcount += 1
                except Exception as e:
                    pass
                    # print(str(e))
        np.random.shuffle(self.training_data)
        np.save("train_data.npy", np.array(self.training_data, dtype=object), allow_pickle=True)
        print("Cats: ", self.catcount)
        print("Dogs: ", self.dogcount)

class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 32, 5)
        self.conv2 = nn.Conv2d(32, 64, 5)
        self.conv3 = nn.Conv2d(64, 128, 5)

        x = torch.randn(50, 50).view(-1, 1, 50, 50)
        self._to_linear = None
        self.convs(x)

        self.fc1 = nn.Linear(self._to_linear, 512)
        self.fc2 = nn.Linear(512, 2)

    def convs(self, x):
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        x = F.max_pool2d(F.relu(self.conv2(x)), (2, 2))
        x = F.max_pool2d(F.relu(self.conv3(x)), (2, 2))
        # print(x[0].shape)

        if self._to_linear is None:
            self._to_linear = x[0].shape[0]*x[0].shape[1]*x[0].shape[2]
        return x
    
    def forward(self, x):
        x = self.convs(x)
        x = x.view(-1, self._to_linear)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.softmax(x, dim=1)
    
net = Net().to(device)
print(net)

if REBUILD_DATA:
    dogsvcats = DogVSCats()
    dogsvcats.make_training_data()

training_data = np.load("train_data.npy", allow_pickle=True)
print(len(training_data))


X = torch.Tensor([i[0] for i in training_data]).view(-1, 50, 50)
X = X/255.0
y = torch.Tensor([i[1] for i in training_data])

VAL_PCT = 0.1
val_size = int(len(X)*VAL_PCT)

train_X = X[:-val_size]
train_y = y[:-val_size]

test_X = X[-val_size:]
test_y = y[-val_size:]

BATCH_SIZE = 100
EPOCHS = 10

def train(net):
    optimizer = optim.Adam(net.parameters(), lr=0.001)
    loss_function = nn.MSELoss()

    for epoch in range(EPOCHS):
        for i in tqdm(range(0, len(train_X), BATCH_SIZE)):
            # print(i, i+BATCH_SIZE)
            batch_X = train_X[i:i+BATCH_SIZE].view(-1, 1, 50, 50)
            batch_y = train_y[i:i+BATCH_SIZE]

            batch_X, batch_y = batch_X.to(device), batch_y.to(device)

            net.zero_grad()

            outputs = net(batch_X)
            loss = loss_function(outputs, batch_y)
            loss.backward()
            optimizer.step()
        print(f"Epoch: {epoch}. Loss: {loss}")

def test(net):
    correct = 0
    total = 0
    with torch.no_grad():
        for i in tqdm(range((len(test_X)))):
            real_class = torch.argmax(test_y[i]).to(device)
            net_out = net(test_X[i].view(-1, 1, 50,50).to(device))[0]
            predicted_class = torch.argmax(net_out)

            if predicted_class == real_class:
                correct += 1
            total += 1

    print("Accuraacy: ", round(correct/total, 10))

train(net)
test(net)

<>:21: SyntaxWarning: invalid escape sequence '\K'
<>:22: SyntaxWarning: invalid escape sequence '\K'
<>:21: SyntaxWarning: invalid escape sequence '\K'
<>:22: SyntaxWarning: invalid escape sequence '\K'
C:\Users\Spieg\AppData\Local\Temp\ipykernel_7272\367283179.py:21: SyntaxWarning: invalid escape sequence '\K'
  CATS = "D:\KMUTT\Year 4th\ENE490 DL\Coding\PetImages\Cat"
C:\Users\Spieg\AppData\Local\Temp\ipykernel_7272\367283179.py:22: SyntaxWarning: invalid escape sequence '\K'
  DOGS = "D:\KMUTT\Year 4th\ENE490 DL\Coding\PetImages\Dog"


runing on the GPU
Net(
  (conv1): Conv2d(1, 32, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(32, 64, kernel_size=(5, 5), stride=(1, 1))
  (conv3): Conv2d(64, 128, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=512, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=2, bias=True)
)
24946


100%|██████████| 225/225 [00:02<00:00, 92.95it/s] 


Epoch: 0. Loss: 0.2363215535879135


100%|██████████| 225/225 [00:02<00:00, 103.36it/s]


...

Epoch: 8. Loss: 0.08847853541374207


100%|██████████| 225/225 [00:02<00:00, 100.82it/s]


Epoch: 9. Loss: 0.09573626518249512


100%|██████████| 2494/2494 [00:01<00:00, 1368.08it/s]

Accuraacy:  0.7477947073


In [13]:
if torch.cuda.is_available():
    device = torch.device("cuda:0")
    print("runing on the GPU")
else:
    device = torch.device("cpu")
    print("runing on the cpu")

net = Net().to(device)
net

runing on the GPU


Net(
  (conv1): Conv2d(1, 32, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(32, 64, kernel_size=(5, 5), stride=(1, 1))
  (conv3): Conv2d(64, 128, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=512, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=2, bias=True)
)